# week 4

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Now access your folder
import os
main_data_folder = '/content/drive/MyDrive/'

Mounted at /content/drive


In [ ]:
!pip install rioxarray

# COMMON DATA LOADER

Loads and prepares data for all three correction models:
  1. Bias Correction Model
  2. Machine Learning Predictor
  3. Statistical Regression Model

This module provides a unified data loading pipeline ensuring consistency
across all modeling approaches.

### LOading master_dataset.csv
**The file master_dataset.csv is produced during the week3.**

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# ============================================================================
# CONFIGURATION
# ============================================================================


DATA_DIR = "/content/drive/MyDrive/GenHack2025/week3_quantitative_metrics"
OUTPUT_DIR = "week4_explanatory_models"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("="*80)
print("WEEK 4: COMMON DATA LOADER")
print("="*80)
print(f"Loading data from: {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

# ============================================================================
# LOAD MASTER DATASET
# ============================================================================

print("\n" + "="*80)
print("STEP 1: LOADING MASTER DATASET")
print("="*80)

# Load main dataset from Week 3
master_file = f"{DATA_DIR}/master_dataset.csv"

if not os.path.exists(master_file):
    raise FileNotFoundError(
        f"\n❌ ERROR: Master dataset not found!\n"
        f"Expected: {master_file}\n\n"
        f"Please run Week 3 analysis first to generate the master dataset."
    )

df_master = pd.read_csv(master_file)

print(f"✓ Loaded master dataset: {len(df_master)} stations")
print(f"  Time period: 2020-2023")
print(f"  Cities: {df_master['city'].nunique()}")
print(f"  Total days analyzed: {df_master['n_days'].sum():,}")

# Display basic statistics
print("\n--- Dataset Overview ---")
print(f"Countries: {df_master['country'].unique()}")
print(f"Cities: {df_master['city'].unique()}")
print(f"\nStation distribution:")
print(df_master['category'].value_counts())
print(f"\nCity distribution:")
print(df_master['city'].value_counts())

# ============================================================================
# DATA QUALITY CHECKS
# ============================================================================

print("\n" + "="*80)
print("STEP 2: DATA QUALITY CHECKS")
print("="*80)

# Check for missing values in critical columns
critical_cols = ['bias', 'mae', 'rmse', 'correlation', 'mean_station_temp',
                 'mean_era5_temp', 'elevation', 'ndvi_mean', 'lat', 'lon']

print("\n--- Missing Values Check ---")
missing_summary = df_master[critical_cols].isnull().sum()
if missing_summary.sum() == 0:
    print("✓ No missing values in critical columns")
else:
    print("⚠️  Missing values detected:")
    print(missing_summary[missing_summary > 0])

# Check for outliers
print("\n--- Outlier Detection ---")
print(f"Bias range: {df_master['bias'].min():.2f}°C to {df_master['bias'].max():.2f}°C")
print(f"RMSE range: {df_master['rmse'].min():.2f}°C to {df_master['rmse'].max():.2f}°C")
print(f"Elevation range: {df_master['elevation'].min():.0f}m to {df_master['elevation'].max():.0f}m")

# Identify potential outliers (>3 std from mean)
bias_outliers = df_master[np.abs(df_master['bias'] - df_master['bias'].mean()) >
                          3 * df_master['bias'].std()]
if len(bias_outliers) > 0:
    print(f"\n⚠️  {len(bias_outliers)} potential bias outliers detected (>3 σ):")
    for _, row in bias_outliers.iterrows():
        print(f"  Station {row['station_id']}: {row['station_name'][:30]} "
              f"(bias: {row['bias']:.2f}°C, elevation: {row['elevation']:.0f}m)")

# ============================================================================
# FEATURE ENGINEERING
# ============================================================================

print("\n" + "="*80)
print("STEP 3: FEATURE ENGINEERING")
print("="*80)

# Create derived features for modeling
df_model = df_master.copy()

# 1. Absolute error
df_model['abs_bias'] = df_model['bias'].abs()

# 2. Relative error (percentage)
df_model['relative_error'] = (df_model['bias'] / df_model['mean_station_temp']) * 100

# 3. Temperature deviation from mean
df_model['temp_deviation'] = df_model['mean_station_temp'] - df_model['mean_station_temp'].mean()

# 4. Elevation categories
df_model['elevation_category'] = pd.cut(
    df_model['elevation'],
    bins=[-np.inf, 100, 300, 600, np.inf],
    labels=['Lowland', 'Low_Hills', 'Hills', 'Mountains']
)

# 5. Distance category
df_model['distance_category'] = pd.cut(
    df_model['distance_to_city_km'],
    bins=[0, 15, 40, 150],
    labels=['Urban', 'Suburban', 'Rural'],
    include_lowest=True
)

# 6. NDVI categories
df_model['ndvi_category'] = pd.cut(
    df_model['ndvi_mean'],
    bins=[-1, 0.2, 0.4, 0.6, 0.8, 1.0],
    labels=['Very_Low', 'Low', 'Medium', 'High', 'Very_High']
)

# 7. Seasonal bias features (from individual season columns)
seasonal_cols = ['bias_Winter', 'bias_Spring', 'bias_Summer', 'bias_Fall']
if all(col in df_model.columns for col in seasonal_cols):
    df_model['seasonal_range'] = (
        df_model[seasonal_cols].max(axis=1) -
        df_model[seasonal_cols].min(axis=1)
    )
    df_model['peak_season_bias'] = df_model[seasonal_cols].max(axis=1)
    print("✓ Created seasonal bias features")
else:
    print("⚠️  Seasonal bias columns not found")

# 8. Coastal vs inland
coastal_threshold = 200  # km
df_model['coastal'] = df_model['distance_to_coast_km'] < coastal_threshold

print(f"\n✓ Created {len([c for c in df_model.columns if c not in df_master.columns])} new features")

# Display new features
print("\n--- New Feature Summary ---")
if 'elevation_category' in df_model.columns:
    print(f"\nElevation categories:")
    print(df_model['elevation_category'].value_counts().sort_index())

if 'ndvi_category' in df_model.columns:
    print(f"\nNDVI categories:")
    print(df_model['ndvi_category'].value_counts().sort_index())

if 'coastal' in df_model.columns:
    print(f"\nCoastal distribution:")
    print(df_model['coastal'].value_counts())

# ============================================================================
# TRAIN-TEST SPLIT
# ============================================================================

print("\n" + "="*80)
print("STEP 4: TRAIN-TEST SPLIT")
print("="*80)

# Strategy: Hold out one city for testing to ensure geographic generalization
test_cities = ['Paris']  # Small dataset, good test case
train_cities = [c for c in df_model['city'].unique() if c not in test_cities]

print(f"\nSplit strategy: Leave-one-city-out")
print(f"  Training cities: {train_cities}")
print(f"  Test city: {test_cities}")

df_train = df_model[df_model['city'].isin(train_cities)].copy()
df_test = df_model[df_model['city'].isin(test_cities)].copy()

print(f"\n✓ Training set: {len(df_train)} stations ({len(df_train)/len(df_model)*100:.1f}%)")
print(f"✓ Test set: {len(df_test)} stations ({len(df_test)/len(df_model)*100:.1f}%)")

# Verify test set has representation across categories
if len(df_test) > 0:
    print(f"\nTest set composition:")
    print(f"  Categories: {df_test['category'].value_counts().to_dict()}")
    print(f"  Elevation range: {df_test['elevation'].min():.0f}m to {df_test['elevation'].max():.0f}m")
    print(f"  Bias range: {df_test['bias'].min():.2f}°C to {df_test['bias'].max():.2f}°C")

# ============================================================================
# PREPARE FEATURE SETS FOR MODELING
# ============================================================================

print("\n" + "="*80)
print("STEP 5: PREPARING FEATURE SETS")
print("="*80)

# Define feature sets for different models

# Basic features (for simple correction model)
basic_features = ['elevation', 'lat', 'lon']

# Extended features (for ML model)
ml_features = [
    'elevation', 'lat', 'lon',
    'mean_station_temp',
    'distance_to_city_km',
    'distance_to_coast_km',
    'ndvi_mean'
]

# Statistical model features (for regression with seasonal data)
stat_features = basic_features + ['mean_station_temp']

print(f"\n✓ Basic features ({len(basic_features)}): {basic_features}")
print(f"✓ ML features ({len(ml_features)}): {ml_features}")
print(f"✓ Statistical features ({len(stat_features)}): {stat_features}")

# Check feature availability
print("\n--- Feature Availability Check ---")
all_features = list(set(basic_features + ml_features + stat_features))
for feat in all_features:
    if feat in df_model.columns:
        print(f"  ✓ {feat}")
    else:
        print(f"  ✗ {feat} - MISSING!")

# ============================================================================
# COMPUTE BASELINE METRICS
# ============================================================================

print("\n" + "="*80)
print("STEP 6: BASELINE METRICS (Before Correction)")
print("="*80)

def calculate_metrics(actual, predicted, label=""):
    """Calculate standard error metrics"""
    errors = predicted - actual

    metrics = {
        'label': label,
        'bias': errors.mean(),
        'mae': np.abs(errors).mean(),
        'rmse': np.sqrt((errors ** 2).mean()),
        'std': errors.std(),
        'r': np.corrcoef(actual, predicted)[0, 1],
        'r2': np.corrcoef(actual, predicted)[0, 1] ** 2,
        'n': len(actual)
    }
    return metrics

# Calculate baseline (ERA5 without any correction)
baseline_train = calculate_metrics(
    df_train['mean_station_temp'],
    df_train['mean_era5_temp'],
    'Training Set Baseline'
)

baseline_test = calculate_metrics(
    df_test['mean_station_temp'],
    df_test['mean_era5_temp'],
    'Test Set Baseline'
)

print("\n--- Baseline Performance (No Corrections) ---")
print(f"\nTraining Set ({len(df_train)} stations):")
print(f"  Bias:        {baseline_train['bias']:>+7.3f}°C")
print(f"  MAE:         {baseline_train['mae']:>7.3f}°C")
print(f"  RMSE:        {baseline_train['rmse']:>7.3f}°C")
print(f"  Correlation: {baseline_train['r']:>7.4f}")
print(f"  R²:          {baseline_train['r2']:>7.4f}")

if len(df_test) > 0:
    print(f"\nTest Set ({len(df_test)} stations):")
    print(f"  Bias:        {baseline_test['bias']:>+7.3f}°C")
    print(f"  MAE:         {baseline_test['mae']:>7.3f}°C")
    print(f"  RMSE:        {baseline_test['rmse']:>7.3f}°C")
    print(f"  Correlation: {baseline_test['r']:>7.4f}")
    print(f"  R²:          {baseline_test['r2']:>7.4f}")

# ============================================================================
# SAVE PROCESSED DATA
# ============================================================================

print("\n" + "="*80)
print("STEP 7: SAVING PROCESSED DATA")
print("="*80)

# Save processed datasets
df_model.to_csv(f"{OUTPUT_DIR}/processed_data_full.csv", index=False)
df_train.to_csv(f"{OUTPUT_DIR}/processed_data_train.csv", index=False)
df_test.to_csv(f"{OUTPUT_DIR}/processed_data_test.csv", index=False)

print(f"\n✓ Saved processed datasets:")
print(f"  - processed_data_full.csv ({len(df_model)} stations)")
print(f"  - processed_data_train.csv ({len(df_train)} stations)")
print(f"  - processed_data_test.csv ({len(df_test)} stations)")

# Save feature lists
feature_info = {
    'basic_features': basic_features,
    'ml_features': ml_features,
    'stat_features': stat_features
}

import json
with open(f"{OUTPUT_DIR}/feature_sets.json", 'w') as f:
    json.dump(feature_info, f, indent=2)

print(f"  - feature_sets.json")

# Save baseline metrics
baseline_metrics = pd.DataFrame([baseline_train, baseline_test])
baseline_metrics.to_csv(f"{OUTPUT_DIR}/baseline_metrics.csv", index=False)
print(f"  - baseline_metrics.csv")

# ============================================================================
# SUMMARY STATISTICS FOR REPORT
# ============================================================================

print("\n" + "="*80)
print("STEP 8: SUMMARY STATISTICS")
print("="*80)

summary_stats = {
    'Total Stations': len(df_model),
    'Training Stations': len(df_train),
    'Test Stations': len(df_test),
    'Total Days Analyzed': int(df_model['n_days'].sum()),
    'Cities': df_model['city'].nunique(),
    'Mean ERA5 Bias': df_model['bias'].mean(),
    'Bias Std Dev': df_model['bias'].std(),
    'Overall RMSE': df_model['rmse'].mean(),
    'Overall MAE': df_model['mae'].mean(),
    'Overall Correlation': df_model['correlation'].mean()
}

print("\n--- Overall Dataset Summary ---")
for key, value in summary_stats.items():
    if isinstance(value, float):
        print(f"{key:<25}: {value:>10.3f}")
    else:
        print(f"{key:<25}: {value:>10}")

# Seasonal summary
if all(f'bias_{s}' in df_model.columns for s in ['Winter', 'Spring', 'Summer', 'Fall']):
    print("\n--- Seasonal Bias Summary ---")
    for season in ['Winter', 'Spring', 'Summer', 'Fall']:
        col = f'bias_{season}'
        seasonal_bias = df_model[col].dropna()
        if len(seasonal_bias) > 0:
            print(f"{season:<10}: {seasonal_bias.mean():>+7.3f} ± {seasonal_bias.std():<6.3f}°C "
                  f"(n={len(seasonal_bias)})")

# Geographic summary
print("\n--- City-Level Summary ---")
city_summary = df_model.groupby('city').agg({
    'bias': ['mean', 'std'],
    'rmse': 'mean',
    'elevation': 'mean',
    'station_id': 'count'
}).round(3)
city_summary.columns = ['Bias_Mean', 'Bias_Std', 'RMSE', 'Elevation_Mean', 'N_Stations']
print(city_summary)

# ============================================================================
# DATA LOADING COMPLETE
# ============================================================================

print("\n" + "="*80)
print("DATA LOADING COMPLETE")
print("="*80)

print("\n✅ All data loaded and prepared successfully!")
print(f"\nReady for modeling:")
print(f"  📊 {len(df_train)} training stations")
print(f"  🧪 {len(df_test)} test stations")
print(f"  📁 {len([c for c in df_model.columns if c not in df_master.columns])} engineered features")
print(f"  📈 Baseline RMSE: {baseline_train['rmse']:.3f}°C")

print(f"\nNext steps:")
print(f"  1. Run Model 1: Bias Correction (Simple)")
print(f"  2. Run Model 2: Machine Learning Predictor")
print(f"  3. Run Model 3: Statistical Regression")

print("\n" + "="*80)

# Make data available for next scripts
print("\n💡 Data objects available for modeling:")
print("   - df_model: Full dataset with engineered features")
print("   - df_train: Training subset")
print("   - df_test: Test subset")
print("   - baseline_train: Training baseline metrics")
print("   - baseline_test: Test baseline metrics")
print("   - ml_features: List of features for ML models")

WEEK 4: COMMON DATA LOADER
Loading data from: /content/drive/MyDrive/GenHack2025/week3_quantitative_metrics
Output directory: week4_explanatory_models

STEP 1: LOADING MASTER DATASET
✓ Loaded master dataset: 286 stations
  Time period: 2020-2023
  Cities: 5
  Total days analyzed: 384,977

--- Dataset Overview ---
Countries: ['DEU' 'FRA' 'ITA' 'POL' 'ESP']
Cities: ['Berlin' 'Paris' 'Milano' 'Warszawa' 'Madrid']

Station distribution:
category
Rural       243
Suburban     22
Urban        21
Name: count, dtype: int64

City distribution:
city
Milano      88
Madrid      87
Berlin      86
Warszawa    20
Paris        5
Name: count, dtype: int64

STEP 2: DATA QUALITY CHECKS

--- Missing Values Check ---
✓ No missing values in critical columns

--- Outlier Detection ---
Bias range: -6.55°C to 2.73°C
RMSE range: 1.11°C to 6.96°C
Elevation range: 1m to 2472m

⚠️  10 potential bias outliers detected (>3 σ):
  Station 17644: LAGDEI (bias: 2.38°C, elevation: 1252m)
  Station 17826: TERUZZI (bias: 2.

## Model 1: Simple, practical correction model based on Week 3 findings:
  - Seasonal adjustment
  - Elevation adjustment
  - Temperature-dependent scaling (optional)

Goal: Reduce ERA5 RMSE by 20-30% with easy-to-apply corrections

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 11

# ============================================================================
# LOAD DATA
# ============================================================================

WEEK3_DIR ="/content/drive/MyDrive/GenHack2025/week3_quantitative_metrics"
WEEK4_DIR = "week4_explanatory_model1"

os.makedirs(WEEK4_DIR, exist_ok=True)

print("="*80)
print("WEEK 4 - MODEL 1: OPERATIONAL BIAS CORRECTION")
print("="*80)

# Load master dataset
df = pd.read_csv(f"{WEEK3_DIR}/master_dataset.csv")
print(f"\nLoaded {len(df)} stations from Week 3")

# ============================================================================
# STEP 1: CALCULATE CORRECTION COMPONENTS
# ============================================================================

print("\n" + "-"*80)
print("STEP 1: CALCULATING CORRECTION COMPONENTS")
print("-"*80)

# Base bias correction (overall mean)
base_correction = -df['bias'].mean()  # Negate because bias is (ERA5 - Observed)
print(f"\n1. Base Correction: +{base_correction:.3f}°C")
print(f"   (Overall mean ERA5 bias: {df['bias'].mean():.3f}°C)")

# Seasonal corrections
print("\n2. Seasonal Corrections:")
seasonal_data = []
for season in ['Winter', 'Spring', 'Summer', 'Fall']:
    col_name = f'bias_{season}'
    if col_name in df.columns:
        season_bias = df[col_name].mean()
        season_correction = base_correction - (-season_bias)  # Adjustment from base
        seasonal_data.append({
            'Season': season,
            'Mean_Bias': season_bias,
            'Correction': -season_bias,
            'Adjustment_from_Base': season_correction
        })
        print(f"   {season:<8}: bias={season_bias:+.3f}°C → correction={-season_bias:+.3f}°C")

df_seasonal = pd.DataFrame(seasonal_data)

# Elevation correction (linear regression)
print("\n3. Elevation Correction:")
valid_elev = df[['elevation', 'bias']].dropna()
slope, intercept, r_value, p_value, std_err = stats.linregress(
    valid_elev['elevation'],
    valid_elev['bias']
)

print(f"   Linear regression: bias = {intercept:.4f} + {slope:.6f} × elevation")
print(f"   R² = {r_value**2:.4f}, p-value = {p_value:.4f}")
print(f"   Elevation coefficient: {slope:.6f}°C/m")

if p_value < 0.05:
    print(f"   ✓ Statistically significant")
    elevation_slope = slope
else:
    print(f"   ✗ Not statistically significant - using zero slope")
    elevation_slope = 0.0

# Temperature-dependent correction (multiplicative effect)
print("\n4. Temperature-Dependent Scaling:")
temp_corr, temp_p = stats.pearsonr(df['bias'].dropna(), df['mean_station_temp'].dropna())
print(f"   Correlation (bias vs temperature): r={temp_corr:.4f}, p={temp_p:.6f}")

if abs(temp_corr) > 0.2 and temp_p < 0.05:
    print(f"   ✓ Multiplicative effect detected")
    temp_slope, temp_intercept, _, _, _ = stats.linregress(
        df['mean_station_temp'].dropna(),
        df['bias'].dropna()
    )
    print(f"   Temperature coefficient: {temp_slope:.4f}°C per °C")
    use_temp_correction = True
else:
    print(f"   ✗ No significant temperature dependence")
    temp_slope = 0.0
    use_temp_correction = False

# ============================================================================
# STEP 2: DEFINE CORRECTION FUNCTION
# ============================================================================

print("\n" + "-"*80)
print("STEP 2: DEFINING CORRECTION FUNCTION")
print("-"*80)

def apply_correction(era5_temp, season, elevation, observed_temp=None, method='full'):
    """
    Apply operational bias correction to ERA5 temperature

    Parameters:
    -----------
    era5_temp : float or array
        Raw ERA5 temperature (°C)
    season : str or array
        Season name ('Winter', 'Spring', 'Summer', 'Fall')
    elevation : float or array
        Station elevation (meters)
    observed_temp : float or array, optional
        Observed temperature for temperature-dependent correction
    method : str
        'base_only' - only base correction
        'seasonal' - base + seasonal
        'full' - base + seasonal + elevation
        'full_temp' - base + seasonal + elevation + temperature-dependent

    Returns:
    --------
    corrected_temp : float or array
        Corrected ERA5 temperature
    """
    correction = np.zeros_like(era5_temp) if isinstance(era5_temp, np.ndarray) else 0.0

    # Base correction (always applied)
    correction += base_correction

    if method in ['seasonal', 'full', 'full_temp']:
        # Seasonal adjustment
        if isinstance(season, str):
            season_adj = df_seasonal[df_seasonal['Season'] == season]['Adjustment_from_Base'].values
            if len(season_adj) > 0:
                correction += season_adj[0]
        else:
            # Array of seasons
            for s in df_seasonal['Season'].unique():
                mask = season == s
                season_adj = df_seasonal[df_seasonal['Season'] == s]['Adjustment_from_Base'].values
                if len(season_adj) > 0:
                    correction = np.where(mask, correction + season_adj[0], correction)

    if method in ['full', 'full_temp']:
        # Elevation adjustment
        correction += elevation_slope * elevation

    if method == 'full_temp' and use_temp_correction and observed_temp is not None:
        # Temperature-dependent adjustment
        correction += temp_slope * observed_temp

    return era5_temp + correction

print("\n✓ Correction function defined")
print(f"   Available methods: 'base_only', 'seasonal', 'full', 'full_temp'")

# ============================================================================
# STEP 3: APPLY CORRECTIONS TO DATASET
# ============================================================================

print("\n" + "-"*80)
print("STEP 3: APPLYING CORRECTIONS")
print("-"*80)

# Determine season for each station (use most common from seasonal data)
def get_dominant_season(row):
    seasons = []
    for season in ['Winter', 'Spring', 'Summer', 'Fall']:
        if f'n_days_{season}' in row.index and row[f'n_days_{season}'] > 0:
            seasons.append((season, row[f'n_days_{season}']))
    if seasons:
        return max(seasons, key=lambda x: x[1])[0]
    return 'Summer'  # Default

df['season'] = df.apply(get_dominant_season, axis=1)

# Apply different correction methods
print("\nApplying correction methods:")

methods = {
    'base_only': 'Base correction only',
    'seasonal': 'Base + Seasonal',
    'full': 'Base + Seasonal + Elevation',
}

if use_temp_correction:
    methods['full_temp'] = 'Base + Seasonal + Elevation + Temperature'

for method_name, description in methods.items():
    df[f'corrected_{method_name}'] = apply_correction(
        df['mean_era5_temp'],
        df['season'],
        df['elevation'],
        df['mean_station_temp'],
        method=method_name
    )

    # Calculate new error
    df[f'error_{method_name}'] = df[f'corrected_{method_name}'] - df['mean_station_temp']
    df[f'abs_error_{method_name}'] = np.abs(df[f'error_{method_name}'])

    print(f"  ✓ {method_name}: {description}")

# ============================================================================
# STEP 4: EVALUATE IMPROVEMENTS
# ============================================================================

print("\n" + "-"*80)
print("STEP 4: EVALUATING CORRECTION PERFORMANCE")
print("-"*80)

# Original errors
original_bias = df['bias'].mean()
original_mae = df['mae'].mean()
original_rmse = df['rmse'].mean()

print(f"\nOriginal ERA5 Performance:")
print(f"  Bias:  {original_bias:+.3f}°C")
print(f"  MAE:   {original_mae:.3f}°C")
print(f"  RMSE:  {original_rmse:.3f}°C")

print(f"\n{'Method':<20} {'Bias (°C)':<12} {'MAE (°C)':<12} {'RMSE (°C)':<12} {'Improvement':<12}")
print("-" * 70)

results = []

for method_name, description in methods.items():
    new_bias = df[f'error_{method_name}'].mean()
    new_mae = df[f'abs_error_{method_name}'].mean()
    new_rmse = np.sqrt((df[f'error_{method_name}']**2).mean())

    improvement = ((original_rmse - new_rmse) / original_rmse) * 100

    print(f"{method_name:<20} {new_bias:>+6.3f}      {new_mae:>6.3f}      "
          f"{new_rmse:>6.3f}      {improvement:>+5.1f}%")

    results.append({
        'Method': description,
        'Method_Code': method_name,
        'Bias': new_bias,
        'MAE': new_mae,
        'RMSE': new_rmse,
        'RMSE_Improvement_%': improvement,
        'Original_RMSE': original_rmse
    })

df_results = pd.DataFrame(results)

# Save results
df_results.to_csv(f"{WEEK4_DIR}/model1_correction_performance.csv", index=False)
print(f"\n✓ Results saved to: {WEEK4_DIR}/model1_correction_performance.csv")

# ============================================================================
# STEP 5: VISUALIZATIONS
# ============================================================================

print("\n" + "-"*80)
print("STEP 5: CREATING VISUALIZATIONS")
print("-"*80)

# Visualization 1: Before vs After Scatter Plots
print("\nCreating Visualization 1: Before/After Comparison...")

fig, axes = plt.subplots(2, 2, figsize=(16, 14))

methods_to_plot = ['base_only', 'seasonal', 'full']
if use_temp_correction:
    methods_to_plot.append('full_temp')

for idx, (ax, method) in enumerate(zip(axes.flat, ['original'] + methods_to_plot[:3])):
    if method == 'original':
        x = df['mean_station_temp']
        y = df['mean_era5_temp']
        title_suffix = 'Original ERA5'
        rmse_val = original_rmse
    else:
        x = df['mean_station_temp']
        y = df[f'corrected_{method}']
        method_desc = methods[method]
        rmse_val = df_results[df_results['Method_Code'] == method]['RMSE'].values[0]
        improvement = df_results[df_results['Method_Code'] == method]['RMSE_Improvement_%'].values[0]
        title_suffix = f'{method_desc}\nRMSE: {rmse_val:.3f}°C ({improvement:+.1f}%)'

    # Scatter plot
    scatter = ax.scatter(x, y, c=df['elevation'], cmap='terrain',
                        s=40, alpha=0.6, edgecolors='black', linewidth=0.5)

    # 1:1 line
    min_val = min(x.min(), y.min())
    max_val = max(x.max(), y.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2,
            label='Perfect Agreement', alpha=0.7)

    # Calculate R²
    correlation = np.corrcoef(x, y)[0, 1]
    r_squared = correlation ** 2

    ax.set_xlabel('Observed Temperature (°C)', fontsize=11)
    ax.set_ylabel('ERA5 Temperature (°C)', fontsize=11)
    ax.set_title(title_suffix, fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=9)

    # Add statistics text
    ax.text(0.05, 0.95, f'R² = {r_squared:.4f}\nRMSE = {rmse_val:.3f}°C',
            transform=ax.transAxes, fontsize=10, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Colorbar
fig.colorbar(scatter, ax=axes.ravel().tolist(), label='Elevation (m)',
             orientation='horizontal', pad=0.05, aspect=40)

fig.suptitle('Model 1: Operational Bias Correction Performance',
             fontsize=15, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(f"{WEEK4_DIR}/viz1_model1_before_after_scatter.png", dpi=300, bbox_inches='tight')
plt.close()
print(f"  ✓ Saved: viz1_model1_before_after_scatter.png")

# Visualization 2: Error Distribution Comparison
print("\nCreating Visualization 2: Error Distribution Comparison...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

for idx, (ax, method) in enumerate(zip(axes.flat, ['original'] + methods_to_plot[:3])):
    if method == 'original':
        errors = df['bias']
        title = 'Original ERA5 Errors'
    else:
        errors = df[f'error_{method}']
        method_desc = methods[method]
        improvement = df_results[df_results['Method_Code'] == method]['RMSE_Improvement_%'].values[0]
        title = f'{method_desc}\n(RMSE improvement: {improvement:+.1f}%)'

    # Histogram
    ax.hist(errors, bins=50, alpha=0.7, color='steelblue', edgecolor='black')

    # Add vertical line at zero
    ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Zero Error')

    # Add mean line
    mean_error = errors.mean()
    ax.axvline(mean_error, color='orange', linestyle='-', linewidth=2,
              label=f'Mean: {mean_error:+.3f}°C')

    ax.set_xlabel('Error (Predicted - Observed) °C', fontsize=11)
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3, axis='y')

fig.suptitle('Model 1: Error Distribution Before and After Corrections',
             fontsize=15, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(f"{WEEK4_DIR}/viz2_model1_error_distributions.png", dpi=300, bbox_inches='tight')
plt.close()
print(f"  ✓ Saved: viz2_model1_error_distributions.png")

# Visualization 3: RMSE Improvement Bar Chart
print("\nCreating Visualization 3: RMSE Improvement Comparison...")

fig, ax = plt.subplots(figsize=(12, 7))

methods_display = df_results['Method'].values
improvements = df_results['RMSE_Improvement_%'].values
rmse_values = df_results['RMSE'].values

x_pos = np.arange(len(methods_display))
colors = ['#e74c3c' if imp < 0 else '#27ae60' for imp in improvements]

bars = ax.bar(x_pos, improvements, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)

# Add value labels on bars
for i, (bar, rmse, imp) in enumerate(zip(bars, rmse_values, improvements)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + (1 if height > 0 else -1),
            f'{imp:+.1f}%\n({rmse:.3f}°C)',
            ha='center', va='bottom' if height > 0 else 'top',
            fontsize=10, fontweight='bold')

ax.axhline(0, color='black', linestyle='-', linewidth=1)
ax.set_ylabel('RMSE Improvement (%)', fontsize=13)
ax.set_xlabel('Correction Method', fontsize=13)
ax.set_title('Model 1: RMSE Improvement by Correction Method\n' +
             f'Original RMSE: {original_rmse:.3f}°C',
             fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(methods_display, rotation=15, ha='right')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(f"{WEEK4_DIR}/viz3_model1_rmse_improvement.png", dpi=300, bbox_inches='tight')
plt.close()
print(f"  ✓ Saved: viz3_model1_rmse_improvement.png")

# Visualization 4: Correction Components
print("\nCreating Visualization 4: Correction Components...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Panel A: Seasonal corrections
ax = axes[0, 0]
seasons = df_seasonal['Season'].values
corrections = df_seasonal['Correction'].values
colors_seasonal = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12']

bars = ax.bar(seasons, corrections, color=colors_seasonal, alpha=0.7,
              edgecolor='black', linewidth=1.5)
for bar, corr in zip(bars, corrections):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.05,
            f'{corr:+.3f}°C', ha='center', va='bottom', fontsize=10, fontweight='bold')

ax.axhline(base_correction, color='red', linestyle='--', linewidth=2,
          label=f'Base: {base_correction:+.3f}°C', alpha=0.7)
ax.set_ylabel('Correction (°C)', fontsize=11)
ax.set_title('A) Seasonal Bias Corrections', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

# Panel B: Elevation effect
ax = axes[0, 1]
ax.scatter(df['elevation'], df['bias'], alpha=0.5, s=30, c='steelblue', edgecolors='black', linewidth=0.5)

# Regression line
x_line = np.linspace(df['elevation'].min(), df['elevation'].max(), 100)
y_line = intercept + slope * x_line
ax.plot(x_line, y_line, 'r-', linewidth=2,
        label=f'y = {intercept:.3f} + {slope:.6f}x\nR² = {r_value**2:.3f}')

ax.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax.set_xlabel('Elevation (m)', fontsize=11)
ax.set_ylabel('ERA5 Bias (°C)', fontsize=11)
ax.set_title('B) Elevation vs Bias Relationship', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Panel C: Temperature dependence
ax = axes[1, 0]
if use_temp_correction:
    ax.scatter(df['mean_station_temp'], df['bias'], alpha=0.5, s=30,
              c='coral', edgecolors='black', linewidth=0.5)

    x_temp = np.linspace(df['mean_station_temp'].min(), df['mean_station_temp'].max(), 100)
    y_temp = temp_intercept + temp_slope * x_temp
    ax.plot(x_temp, y_temp, 'r-', linewidth=2,
            label=f'y = {temp_intercept:.3f} + {temp_slope:.4f}x\nr = {temp_corr:.3f}')

    ax.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    ax.set_xlabel('Mean Observed Temperature (°C)', fontsize=11)
    ax.set_ylabel('ERA5 Bias (°C)', fontsize=11)
    ax.set_title('C) Temperature Dependence (Multiplicative Effect)', fontsize=12, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
else:
    ax.text(0.5, 0.5, 'No significant\ntemperature dependence\ndetected',
            ha='center', va='center', transform=ax.transAxes, fontsize=14)
    ax.set_title('C) Temperature Dependence', fontsize=12, fontweight='bold')

# Panel D: Summary table
ax = axes[1, 1]
ax.axis('tight')
ax.axis('off')

summary_data = [
    ['Component', 'Value', 'Significance'],
    ['Base Correction', f'{base_correction:+.3f}°C', '✓ Always applied'],
    ['Seasonal Range', f'{corrections.max() - corrections.min():.3f}°C',
     f'✓ p<0.001' if len(df_seasonal) > 0 else 'N/A'],
    ['Elevation Slope', f'{elevation_slope:.6f}°C/m',
     '✓ Significant' if p_value < 0.05 else '✗ Not significant'],
    ['Temp Coefficient', f'{temp_slope:.4f}°C/°C' if use_temp_correction else 'N/A',
     '✓ Multiplicative' if use_temp_correction else '✗ Not detected'],
    ['', '', ''],
    ['Best Method', methods[df_results.iloc[-1]['Method_Code']], ''],
    ['RMSE Improvement', f"{df_results.iloc[-1]['RMSE_Improvement_%']:+.1f}%", ''],
    ['Final RMSE', f"{df_results.iloc[-1]['RMSE']:.3f}°C",
     f'(was {original_rmse:.3f}°C)']
]

table = ax.table(cellText=summary_data, cellLoc='left', loc='center',
                colWidths=[0.35, 0.25, 0.4])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2.5)

# Style header row
for i in range(3):
    table[(0, i)].set_facecolor('#34495e')
    table[(0, i)].set_text_props(weight='bold', color='white')

# Style separator row
for i in range(3):
    table[(5, i)].set_facecolor('#ecf0f1')

ax.set_title('D) Correction Summary', fontsize=12, fontweight='bold', pad=20)

fig.suptitle('Model 1: Correction Components Analysis',
             fontsize=15, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(f"{WEEK4_DIR}/viz4_model1_correction_components.png", dpi=300, bbox_inches='tight')
plt.close()
print(f"  ✓ Saved: viz4_model1_correction_components.png")

# Visualization 5: Error Reduction Map by City
print("\nCreating Visualization 5: Error Reduction by City...")

fig, ax = plt.subplots(figsize=(12, 7))

# Use best method
best_method = df_results.iloc[-1]['Method_Code']

city_improvements = []
for city in sorted(df['city'].unique()):
    city_data = df[df['city'] == city]
    original_city_rmse = city_data['rmse'].mean()
    corrected_city_rmse = np.sqrt((city_data[f'error_{best_method}']**2).mean())
    improvement = ((original_city_rmse - corrected_city_rmse) / original_city_rmse) * 100

    city_improvements.append({
        'City': city,
        'Original_RMSE': original_city_rmse,
        'Corrected_RMSE': corrected_city_rmse,
        'Improvement_%': improvement,
        'N_Stations': len(city_data)
    })

df_city_imp = pd.DataFrame(city_improvements)
df_city_imp = df_city_imp.sort_values('Improvement_%', ascending=False)

x_pos = np.arange(len(df_city_imp))
colors_city = ['#27ae60' if imp > 0 else '#e74c3c' for imp in df_city_imp['Improvement_%']]

bars = ax.bar(x_pos, df_city_imp['Improvement_%'], color=colors_city,
              alpha=0.7, edgecolor='black', linewidth=1.5)

for i, (bar, imp, orig, corr) in enumerate(zip(bars,
                                                 df_city_imp['Improvement_%'],
                                                 df_city_imp['Original_RMSE'],
                                                 df_city_imp['Corrected_RMSE'])):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 1,
            f'{imp:+.1f}%\n{orig:.2f}→{corr:.2f}°C',
            ha='center', va='bottom', fontsize=9, fontweight='bold')

ax.axhline(0, color='black', linestyle='-', linewidth=1)
ax.set_ylabel('RMSE Improvement (%)', fontsize=13)
ax.set_xlabel('City', fontsize=13)
ax.set_title(f'Model 1: RMSE Improvement by City\nUsing {methods[best_method]}',
             fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(df_city_imp['City'], rotation=0)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(f"{WEEK4_DIR}/viz5_model1_city_improvements.png", dpi=300, bbox_inches='tight')
plt.close()
print(f"  ✓ Saved: viz5_model1_city_improvements.png")

# Save city improvements
df_city_imp.to_csv(f"{WEEK4_DIR}/model1_city_improvements.csv", index=False)

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*80)
print("MODEL 1: OPERATIONAL BIAS CORRECTION - COMPLETE")
print("="*80)

print("\n📊 PERFORMANCE SUMMARY:")
print(f"  Original RMSE: {original_rmse:.3f}°C")

best_result = df_results.iloc[-1]
print(f"\n  Best Method: {best_result['Method']}")
print(f"  Corrected RMSE: {best_result['RMSE']:.3f}°C")
print(f"  Improvement: {best_result['RMSE_Improvement_%']:+.1f}%")
print(f"  New Bias: {best_result['Bias']:+.3f}°C (was {original_bias:+.3f}°C)")

print("\n📁 OUTPUTS SAVED:")
print(f"  1. Performance metrics: model1_correction_performance.csv")
print(f"  2. City improvements: model1_city_improvements.csv")
print(f"  3. Before/after scatter: viz1_model1_before_after_scatter.png")
print(f"  4. Error distributions: viz2_model1_error_distributions.png")
print(f"  5. RMSE improvement: viz3_model1_rmse_improvement.png")
print(f"  6. Correction components: viz4_model1_correction_components.png")
print(f"  7. City improvements: viz5_model1_city_improvements.png")

print("\n✓ Model 1 analysis complete!")

WEEK 4 - MODEL 1: OPERATIONAL BIAS CORRECTION

Loaded 286 stations from Week 3

--------------------------------------------------------------------------------
STEP 1: CALCULATING CORRECTION COMPONENTS
--------------------------------------------------------------------------------

1. Base Correction: +1.254°C
   (Overall mean ERA5 bias: -1.254°C)

2. Seasonal Corrections:
   Winter  : bias=-0.780°C → correction=+0.780°C
   Spring  : bias=-1.394°C → correction=+1.394°C
   Summer  : bias=-1.689°C → correction=+1.689°C
   Fall    : bias=-1.005°C → correction=+1.005°C

3. Elevation Correction:
   Linear regression: bias = -1.2823 + 0.000061 × elevation
   R² = 0.0006, p-value = 0.6888
   Elevation coefficient: 0.000061°C/m
   ✗ Not statistically significant - using zero slope

4. Temperature-Dependent Scaling:
   Correlation (bias vs temperature): r=-0.3285, p=0.000000
   ✓ Multiplicative effect detected
   Temperature coefficient: -0.1278°C per °C

-------------------------------------

## Model 2: Advanced ML approach to predict and correct ERA5 errors:
  - Random Forest Regressor (primary model)
  - Gradient Boosting (alternative)
  - Feature importance analysis
  - Cross-validation
  - Hold-out test set validation
  - Spatial cross-validation (leave-one-city-out)

Goal: Learn complex non-linear patterns in ERA5 errors

In [4]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 11

# ============================================================================
# LOAD DATA
# ============================================================================

WEEK3_DIR ="/content/drive/MyDrive/GenHack2025/week3_quantitative_metrics"
WEEK4_DIR = "week4_explanatory_model2"
os.makedirs(WEEK4_DIR, exist_ok=True)

print("="*80)
print("WEEK 4 - MODEL 2: MACHINE LEARNING ERROR PREDICTOR")
print("="*80)

# Load master dataset
df = pd.read_csv(f"{WEEK3_DIR}/master_dataset.csv")
print(f"\nLoaded {len(df)} stations from Week 3")

# ============================================================================
# STEP 1: FEATURE ENGINEERING
# ============================================================================

print("\n" + "-"*80)
print("STEP 1: FEATURE ENGINEERING")
print("-"*80)

# Define features
feature_columns = [
    'ndvi_mean',
    'elevation',
    'lat',
    'lon',
    'distance_to_city_km',
    'distance_to_coast_km',
    'mean_station_temp'
]

# Add season encoding (one-hot)
print("\nAdding seasonal features...")
def get_dominant_season(row):
    seasons = []
    for season in ['Winter', 'Spring', 'Summer', 'Fall']:
        if f'n_days_{season}' in row.index and row[f'n_days_{season}'] > 0:
            seasons.append((season, row[f'n_days_{season}']))
    if seasons:
        return max(seasons, key=lambda x: x[1])[0]
    return 'Summer'

df['season'] = df.apply(get_dominant_season, axis=1)

# One-hot encode season
season_dummies = pd.get_dummies(df['season'], prefix='season')
df = pd.concat([df, season_dummies], axis=1)

# Add season columns to features
season_features = [col for col in df.columns if col.startswith('season_')]
feature_columns.extend(season_features)

print(f"  ✓ Total features: {len(feature_columns)}")
print(f"    Base features: 7")
print(f"    Season features: {len(season_features)}")

# Target variable
target_column = 'bias'

# Prepare data
df_clean = df[feature_columns + [target_column, 'city', 'category']].dropna()
print(f"\n✓ Clean dataset: {len(df_clean)} samples")

X = df_clean[feature_columns].values
y = df_clean[target_column].values

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

# Feature statistics
print(f"\nFeature ranges:")
for i, col in enumerate(feature_columns[:7]):  # Skip one-hot encoded
    print(f"  {col:<25}: [{X[:, i].min():.3f}, {X[:, i].max():.3f}]")

# ============================================================================
# STEP 2: TRAIN-TEST SPLIT
# ============================================================================

print("\n" + "-"*80)
print("STEP 2: TRAIN-TEST SPLIT")
print("-"*80)

# Random 80-20 split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nTraining set: {len(X_train)} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"Test set: {len(X_test)} samples ({len(X_test)/len(X)*100:.1f}%)")

# Store indices for later
train_idx = df_clean.index[:(len(X_train))]
test_idx = df_clean.index[len(X_train):]

# ============================================================================
# STEP 3: TRAIN MODELS
# ============================================================================

print("\n" + "-"*80)
print("STEP 3: TRAINING MODELS")
print("-"*80)

# Model 1: Random Forest
print("\n1. Random Forest Regressor")
print("   Hyperparameters:")
rf_params = {
    'n_estimators': 200,
    'max_depth': 15,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'random_state': 42,
    'n_jobs': -1
}
for param, value in rf_params.items():
    print(f"     {param}: {value}")

rf_model = RandomForestRegressor(**rf_params)
rf_model.fit(X_train, y_train)
print("   ✓ Training complete")

# Model 2: Gradient Boosting
print("\n2. Gradient Boosting Regressor")
print("   Hyperparameters:")
gb_params = {
    'n_estimators': 150,
    'max_depth': 8,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'random_state': 42
}
for param, value in gb_params.items():
    print(f"     {param}: {value}")

gb_model = GradientBoostingRegressor(**gb_params)
gb_model.fit(X_train, y_train)
print("   ✓ Training complete")

# ============================================================================
# STEP 4: EVALUATE ON TEST SET
# ============================================================================

print("\n" + "-"*80)
print("STEP 4: TEST SET EVALUATION")
print("-"*80)

models = {
    'Random Forest': rf_model,
    'Gradient Boosting': gb_model
}

test_results = []

for model_name, model in models.items():
    print(f"\n{model_name}:")

    # Predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Metrics - Training
    train_r2 = r2_score(y_train, y_pred_train)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    train_mae = mean_absolute_error(y_train, y_pred_train)

    # Metrics - Test
    test_r2 = r2_score(y_test, y_pred_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    test_mae = mean_absolute_error(y_test, y_pred_test)

    print(f"  Training: R²={train_r2:.4f}, RMSE={train_rmse:.3f}°C, MAE={train_mae:.3f}°C")
    print(f"  Test:     R²={test_r2:.4f}, RMSE={test_rmse:.3f}°C, MAE={test_mae:.3f}°C")

    # Check for overfitting
    if train_r2 - test_r2 > 0.1:
        print(f"  ⚠️  Warning: Possible overfitting (R² gap: {train_r2 - test_r2:.3f})")
    else:
        print(f"  ✓ Good generalization")

    test_results.append({
        'Model': model_name,
        'Train_R2': train_r2,
        'Train_RMSE': train_rmse,
        'Train_MAE': train_mae,
        'Test_R2': test_r2,
        'Test_RMSE': test_rmse,
        'Test_MAE': test_mae,
        'Predictions_Train': y_pred_train,
        'Predictions_Test': y_pred_test
    })

df_test_results = pd.DataFrame(test_results)

# ============================================================================
# STEP 5: CROSS-VALIDATION
# ============================================================================

print("\n" + "-"*80)
print("STEP 5: CROSS-VALIDATION (5-FOLD)")
print("-"*80)

cv_results = []

for model_name, model in models.items():
    print(f"\n{model_name}:")

    # 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    cv_r2 = cross_val_score(model, X, y, cv=kf, scoring='r2', n_jobs=-1)
    cv_neg_mse = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error', n_jobs=-1)
    cv_rmse = np.sqrt(-cv_neg_mse)

    print(f"  R² scores: {cv_r2}")
    print(f"  Mean R²: {cv_r2.mean():.4f} (±{cv_r2.std():.4f})")
    print(f"  RMSE scores: {cv_rmse}")
    print(f"  Mean RMSE: {cv_rmse.mean():.3f}°C (±{cv_rmse.std():.3f})")

    cv_results.append({
        'Model': model_name,
        'CV_R2_Mean': cv_r2.mean(),
        'CV_R2_Std': cv_r2.std(),
        'CV_RMSE_Mean': cv_rmse.mean(),
        'CV_RMSE_Std': cv_rmse.std()
    })

df_cv_results = pd.DataFrame(cv_results)

# ============================================================================
# STEP 6: SPATIAL CROSS-VALIDATION (LEAVE-ONE-CITY-OUT)
# ============================================================================

print("\n" + "-"*80)
print("STEP 6: SPATIAL CROSS-VALIDATION (LEAVE-ONE-CITY-OUT)")
print("-"*80)

cities = df_clean['city'].unique()
print(f"\nTesting generalization across {len(cities)} cities")

spatial_cv_results = []

for test_city in cities:
    print(f"\n  Held-out city: {test_city}")

    # Split by city
    train_mask = df_clean['city'] != test_city
    test_mask = df_clean['city'] == test_city

    X_train_cv = X[train_mask]
    y_train_cv = y[train_mask]
    X_test_cv = X[test_mask]
    y_test_cv = y[test_mask]

    print(f"    Train: {len(X_train_cv)} stations from {len(cities)-1} cities")
    print(f"    Test:  {len(X_test_cv)} stations from {test_city}")

    for model_name, model in models.items():
        # Train on other cities
        model_cv = model.__class__(**model.get_params())
        model_cv.fit(X_train_cv, y_train_cv)

        # Predict on held-out city
        y_pred_cv = model_cv.predict(X_test_cv)

        # Metrics
        r2_cv = r2_score(y_test_cv, y_pred_cv)
        rmse_cv = np.sqrt(mean_squared_error(y_test_cv, y_pred_cv))

        spatial_cv_results.append({
            'Model': model_name,
            'Held_Out_City': test_city,
            'N_Test': len(y_test_cv),
            'R2': r2_cv,
            'RMSE': rmse_cv
        })

df_spatial_cv = pd.DataFrame(spatial_cv_results)

print("\n  Summary by model:")
for model_name in models.keys():
    model_results = df_spatial_cv[df_spatial_cv['Model'] == model_name]
    print(f"\n  {model_name}:")
    print(f"    Mean R²: {model_results['R2'].mean():.4f} (±{model_results['R2'].std():.4f})")
    print(f"    Mean RMSE: {model_results['RMSE'].mean():.3f}°C (±{model_results['RMSE'].std():.3f})")

# ============================================================================
# STEP 7: FEATURE IMPORTANCE
# ============================================================================

print("\n" + "-"*80)
print("STEP 7: FEATURE IMPORTANCE ANALYSIS")
print("-"*80)

# Use Random Forest for feature importance
feature_importance = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(f"{'Rank':<6} {'Feature':<25} {'Importance':<12} {'Relative %':<12}")
print("-" * 60)

total_importance = feature_importance['Importance'].sum()
for idx, row in feature_importance.head(10).iterrows():
    rank = feature_importance.index.get_loc(idx) + 1
    relative_pct = (row['Importance'] / total_importance) * 100
    print(f"{rank:<6} {row['Feature']:<25} {row['Importance']:>8.4f}    {relative_pct:>6.2f}%")

# Save feature importance
feature_importance.to_csv(f"{WEEK4_DIR}/model2_feature_importance.csv", index=False)


WEEK 4 - MODEL 2: MACHINE LEARNING ERROR PREDICTOR

Loaded 286 stations from Week 3

--------------------------------------------------------------------------------
STEP 1: FEATURE ENGINEERING
--------------------------------------------------------------------------------

Adding seasonal features...
  ✓ Total features: 11
    Base features: 7
    Season features: 4

✓ Clean dataset: 286 samples

Feature matrix shape: (286, 11)
Target vector shape: (286,)

Feature ranges:
  ndvi_mean                : [-0.258, 0.737]
  elevation                : [1.000, 2472.000]
  lat                      : [39.687, 53.746]
  lon                      : [-5.498, 22.550]
  distance_to_city_km      : [1.730, 148.967]
  distance_to_coast_km     : [173.347, 1026.540]
  mean_station_temp        : [3.883, 24.728]

--------------------------------------------------------------------------------
STEP 2: TRAIN-TEST SPLIT
--------------------------------------------------------------------------------

Training

In [ ]:
df_clean.columns

Index(['ndvi_mean', 'elevation', 'lat', 'lon', 'distance_to_city_km',
       'distance_to_coast_km', 'mean_station_temp', 'season_Fall',
       'season_Spring', 'season_Summer', 'season_Winter', 'bias', 'city',
       'category'],
      dtype='object')

In [5]:

# ============================================================================
# STEP 8: APPLY CORRECTIONS
# ============================================================================

print("\n" + "-"*80)
print("STEP 8: APPLYING ML CORRECTIONS")
print("-"*80)

# Use best model (Random Forest)
best_model = rf_model
best_model_name = 'Random Forest'

print(f"\nUsing {best_model_name} for corrections")

# Predict errors for all data
y_pred_all = best_model.predict(X)

# Apply corrections to ERA5
df_clean['predicted_error'] = y_pred_all
df_clean['era5_corrected_ml'] = df_clean['mean_station_temp'] - df_clean['predicted_error']
df_clean['error_ml'] = df_clean['era5_corrected_ml'] - df_clean['mean_station_temp']

# Calculate improvement
original_rmse = df['rmse'].mean()
ml_rmse = np.sqrt((df_clean['error_ml']**2).mean())
improvement = ((original_rmse - ml_rmse) / original_rmse) * 100

print(f"\nOriginal ERA5 RMSE: {original_rmse:.3f}°C")
print(f"ML-Corrected RMSE: {ml_rmse:.3f}°C")
print(f"Improvement: {improvement:+.1f}%")

# Save corrected dataset
df_clean.to_csv(f"{WEEK4_DIR}/model2_corrected_dataset.csv", index=False)

# ============================================================================
# VISUALIZATIONS
# ============================================================================

print("\n" + "-"*80)
print("STEP 9: CREATING VISUALIZATIONS")
print("-"*80)

# Viz 1: Model Comparison - Performance Metrics
print("\nCreating Viz 1: Model Performance Comparison...")

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

metrics = ['Train_R2', 'Test_R2', 'CV_R2_Mean']
titles = ['Training R²', 'Test R²', 'Cross-Validation R²']

for ax, metric, title in zip(axes, metrics, titles):
    if metric in df_test_results.columns:
        values = df_test_results[metric].values
    else:
        values = df_cv_results[metric].values

    bars = ax.bar(df_test_results['Model'], values,
                  color=['#3498db', '#e74c3c'], alpha=0.7, edgecolor='black', linewidth=1.5)

    for bar, val in zip(bars, values):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{val:.4f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

    ax.set_ylabel('R² Score', fontsize=12)
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.set_ylim([0, max(values) * 1.15])
    ax.grid(True, alpha=0.3, axis='y')

fig.suptitle('Model 2: Machine Learning Performance Comparison',
             fontsize=15, fontweight='bold')
plt.tight_layout()
plt.savefig(f"{WEEK4_DIR}/viz1_model2_performance_comparison.png", dpi=300, bbox_inches='tight')
plt.close()
print("  ✓ Saved: viz1_model2_performance_comparison.png")

# Viz 2: Predicted vs Actual (Train and Test)
print("\nCreating Viz 2: Predicted vs Actual Errors...")

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

rf_result = df_test_results[df_test_results['Model'] == 'Random Forest'].iloc[0]

# Training set
ax = axes[0]
scatter = ax.scatter(y_train, rf_result['Predictions_Train'],
                    c=X_train[:, feature_columns.index('elevation')],
                    cmap='terrain', s=50, alpha=0.6, edgecolors='black', linewidth=0.5)

min_val = min(y_train.min(), rf_result['Predictions_Train'].min())
max_val = max(y_train.max(), rf_result['Predictions_Train'].max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2,
        label='Perfect Prediction', alpha=0.7)

ax.set_xlabel('Actual ERA5 Bias (°C)', fontsize=12)
ax.set_ylabel('Predicted ERA5 Bias (°C)', fontsize=12)
ax.set_title(f'Training Set (n={len(y_train)})\nR²={rf_result["Train_R2"]:.4f}, RMSE={rf_result["Train_RMSE"]:.3f}°C',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Test set
ax = axes[1]
scatter = ax.scatter(y_test, rf_result['Predictions_Test'],
                    c=X_test[:, feature_columns.index('elevation')],
                    cmap='terrain', s=50, alpha=0.6, edgecolors='black', linewidth=0.5)

min_val = min(y_test.min(), rf_result['Predictions_Test'].min())
max_val = max(y_test.max(), rf_result['Predictions_Test'].max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2,
        label='Perfect Prediction', alpha=0.7)

ax.set_xlabel('Actual ERA5 Bias (°C)', fontsize=12)
ax.set_ylabel('Predicted ERA5 Bias (°C)', fontsize=12)
ax.set_title(f'Test Set (n={len(y_test)})\nR²={rf_result["Test_R2"]:.4f}, RMSE={rf_result["Test_RMSE"]:.3f}°C',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

fig.colorbar(scatter, ax=axes, label='Elevation (m)', orientation='horizontal', pad=0.1)

fig.suptitle('Model 2: Random Forest Prediction Accuracy',
             fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(f"{WEEK4_DIR}/viz2_model2_predictions.png", dpi=300, bbox_inches='tight')
plt.close()
print("  ✓ Saved: viz2_model2_predictions.png")

# Viz 3: Feature Importance
print("\nCreating Viz 3: Feature Importance...")

fig, ax = plt.subplots(figsize=(12, 8))

top_features = feature_importance.head(10)
colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(top_features)))

bars = ax.barh(range(len(top_features)), top_features['Importance'],
               color=colors, edgecolor='black', linewidth=1.5)

ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features['Feature'])
ax.invert_yaxis()
ax.set_xlabel('Importance Score', fontsize=13)
ax.set_title('Model 2: Top 10 Feature Importances (Random Forest)',
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# Add values
for i, (bar, imp) in enumerate(zip(bars, top_features['Importance'])):
    width = bar.get_width()
    relative_pct = (imp / total_importance) * 100
    ax.text(width + 0.005, bar.get_y() + bar.get_height()/2.,
            f'{imp:.4f} ({relative_pct:.1f}%)',
            va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig(f"{WEEK4_DIR}/viz3_model2_feature_importance.png", dpi=300, bbox_inches='tight')
plt.close()
print("  ✓ Saved: viz3_model2_feature_importance.png")

# Viz 4: Spatial Cross-Validation Results
print("\nCreating Viz 4: Spatial Cross-Validation...")

fig, ax = plt.subplots(figsize=(12, 7))

rf_spatial = df_spatial_cv[df_spatial_cv['Model'] == 'Random Forest']
x_pos = np.arange(len(rf_spatial))

bars = ax.bar(x_pos, rf_spatial['RMSE'],
              color=['#27ae60' if r < original_rmse else '#e74c3c'
                     for r in rf_spatial['RMSE']],
              alpha=0.7, edgecolor='black', linewidth=1.5)

# Add original RMSE line
ax.axhline(original_rmse, color='red', linestyle='--', linewidth=2,
          label=f'Original ERA5 RMSE: {original_rmse:.3f}°C', alpha=0.7)

# Add values
for bar, rmse, r2 in zip(bars, rf_spatial['RMSE'], rf_spatial['R2']):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.05,
            f'{rmse:.3f}°C\nR²={r2:.3f}',
            ha='center', va='bottom', fontsize=9, fontweight='bold')

ax.set_xticks(x_pos)
ax.set_xticklabels(rf_spatial['Held_Out_City'], rotation=0)
ax.set_ylabel('RMSE (°C)', fontsize=13)
ax.set_xlabel('Held-Out City', fontsize=13)
ax.set_title('Model 2: Leave-One-City-Out Cross-Validation\nRandom Forest Performance',
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(f"{WEEK4_DIR}/viz4_model2_spatial_cv.png", dpi=300, bbox_inches='tight')
plt.close()
print("  ✓ Saved: viz4_model2_spatial_cv.png")

# Viz 5: Before vs After Correction
print("\nCreating Viz 5: Before/After ML Correction...")

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Original
ax = axes[0]
scatter = ax.scatter(df_clean['mean_station_temp'], df_clean['mean_station_temp'],
                    c=df_clean['elevation'], cmap='terrain', s=50, alpha=0.6,
                    edgecolors='black', linewidth=0.5)

min_val = min(df_clean['mean_station_temp'].min(), df_clean['mean_station_temp'].min())
max_val = max(df_clean['mean_station_temp'].max(), df_clean['mean_station_temp'].max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2,
        label='Perfect Agreement', alpha=0.7)

ax.set_xlabel('Observed Temperature (°C)', fontsize=12)
ax.set_ylabel('ERA5 Temperature (°C)', fontsize=12)
ax.set_title(f'Original ERA5\nRMSE: {original_rmse:.3f}°C',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# ML Corrected
ax = axes[1]
scatter = ax.scatter(df_clean['mean_station_temp'], df_clean['era5_corrected_ml'],
                    c=df_clean['elevation'], cmap='terrain', s=50, alpha=0.6,
                    edgecolors='black', linewidth=0.5)

min_val = min(df_clean['mean_station_temp'].min(), df_clean['era5_corrected_ml'].min())
max_val = max(df_clean['mean_station_temp'].max(), df_clean['era5_corrected_ml'].max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2,
        label='Perfect Agreement', alpha=0.7)

ax.set_xlabel('Observed Temperature (°C)', fontsize=12)
ax.set_ylabel('Corrected ERA5 Temperature (°C)', fontsize=12)
ax.set_title(f'ML-Corrected ERA5\nRMSE: {ml_rmse:.3f}°C ({improvement:+.1f}%)',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

fig.colorbar(scatter, ax=axes, label='Elevation (m)', orientation='horizontal', pad=0.1)

fig.suptitle('Model 2: Impact of Machine Learning Correction',
             fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(f"{WEEK4_DIR}/viz5_model2_correction_impact.png", dpi=300, bbox_inches='tight')
plt.close()
print("  ✓ Saved: viz5_model2_correction_impact.png")

# Viz 6: Residuals Analysis
print("\nCreating Viz 6: Residuals Analysis...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

rf_residuals_train = y_train - rf_result['Predictions_Train']
rf_residuals_test = y_test - rf_result['Predictions_Test']

# Training residuals vs predicted
ax = axes[0, 0]
ax.scatter(rf_result['Predictions_Train'], rf_residuals_train,
          alpha=0.5, s=30, c='steelblue', edgecolors='black', linewidth=0.5)
ax.axhline(0, color='red', linestyle='--', linewidth=2)
ax.set_xlabel('Predicted Bias (°C)', fontsize=11)
ax.set_ylabel('Residuals (°C)', fontsize=11)
ax.set_title('A) Training Residuals vs Predicted', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Test residuals vs predicted
ax = axes[0, 1]
ax.scatter(rf_result['Predictions_Test'], rf_residuals_test,
          alpha=0.5, s=30, c='coral', edgecolors='black', linewidth=0.5)
ax.axhline(0, color='red', linestyle='--', linewidth=2)
ax.set_xlabel('Predicted Bias (°C)', fontsize=11)
ax.set_ylabel('Residuals (°C)', fontsize=11)
ax.set_title('B) Test Residuals vs Predicted', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Training residuals histogram
ax = axes[1, 0]
ax.hist(rf_residuals_train, bins=40, alpha=0.7, color='steelblue', edgecolor='black')
ax.axvline(0, color='red', linestyle='--', linewidth=2)
ax.set_xlabel('Residuals (°C)', fontsize=11)
ax.set_ylabel('Frequency', fontsize=11)
ax.set_title(f'C) Training Residuals Distribution\nMean: {rf_residuals_train.mean():+.4f}°C, Std: {rf_residuals_train.std():.4f}°C',
             fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Test residuals histogram
ax = axes[1, 1]
ax.hist(rf_residuals_test, bins=40, alpha=0.7, color='coral', edgecolor='black')
ax.axvline(0, color='red', linestyle='--', linewidth=2)
ax.set_xlabel('Residuals (°C)', fontsize=11)
ax.set_ylabel('Frequency', fontsize=11)
ax.set_title(f'D) Test Residuals Distribution\nMean: {rf_residuals_test.mean():+.4f}°C, Std: {rf_residuals_test.std():.4f}°C',
             fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

fig.suptitle('Model 2: Residuals Analysis (Random Forest)',
             fontsize=15, fontweight='bold')
plt.tight_layout()
plt.savefig(f"{WEEK4_DIR}/viz6_model2_residuals.png", dpi=300, bbox_inches='tight')
plt.close()
print("  ✓ Saved: viz6_model2_residuals.png")

# ============================================================================
# SAVE ALL RESULTS
# ============================================================================

print("\n" + "-"*80)
print("SAVING RESULTS")
print("-"*80)

# Save test results
df_test_results[['Model', 'Train_R2', 'Train_RMSE', 'Train_MAE',
                 'Test_R2', 'Test_RMSE', 'Test_MAE']].to_csv(
    f"{WEEK4_DIR}/model2_test_results.csv", index=False
)
print("  ✓ Test results: model2_test_results.csv")

# Save CV results
df_cv_results.to_csv(f"{WEEK4_DIR}/model2_cv_results.csv", index=False)
print("  ✓ CV results: model2_cv_results.csv")

# Save spatial CV results
df_spatial_cv.to_csv(f"{WEEK4_DIR}/model2_spatial_cv_results.csv", index=False)
print("  ✓ Spatial CV results: model2_spatial_cv_results.csv")

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*80)
print("MODEL 2: MACHINE LEARNING ERROR PREDICTOR - COMPLETE")
print("="*80)

print("\n📊 BEST MODEL: Random Forest")
print(f"\n  Training Performance:")
print(f"    R²:   {rf_result['Train_R2']:.4f}")
print(f"    RMSE: {rf_result['Train_RMSE']:.3f}°C")
print(f"    MAE:  {rf_result['Train_MAE']:.3f}°C")

print(f"\n  Test Performance:")
print(f"    R²:   {rf_result['Test_R2']:.4f}")
print(f"    RMSE: {rf_result['Test_RMSE']:.3f}°C")
print(f"    MAE:  {rf_result['Test_MAE']:.3f}°C")

print(f"\n  Cross-Validation:")
rf_cv = df_cv_results[df_cv_results['Model'] == 'Random Forest'].iloc[0]
print(f"    R²:   {rf_cv['CV_R2_Mean']:.4f} (±{rf_cv['CV_R2_Std']:.4f})")
print(f"    RMSE: {rf_cv['CV_RMSE_Mean']:.3f}°C (±{rf_cv['CV_RMSE_Std']:.3f})")

print(f"\n  Spatial Cross-Validation:")
rf_spatial_mean = df_spatial_cv[df_spatial_cv['Model'] == 'Random Forest']
print(f"    Mean R²:   {rf_spatial_mean['R2'].mean():.4f} (±{rf_spatial_mean['R2'].std():.4f})")
print(f"    Mean RMSE: {rf_spatial_mean['RMSE'].mean():.3f}°C (±{rf_spatial_mean['RMSE'].std():.3f})")

print(f"\n🎯 CORRECTION PERFORMANCE:")
print(f"  Original ERA5 RMSE: {original_rmse:.3f}°C")
print(f"  ML-Corrected RMSE:  {ml_rmse:.3f}°C")
print(f"  Improvement:        {improvement:+.1f}%")

print(f"\n🏆 TOP 3 FEATURES:")
for i, row in feature_importance.head(3).iterrows():
    pct = (row['Importance'] / total_importance) * 100
    print(f"  {i+1}. {row['Feature']:<25} ({pct:.1f}%)")

print("\n📁 OUTPUTS SAVED:")
print("  1. model2_feature_importance.csv - Feature rankings")
print("  2. model2_corrected_dataset.csv - Full corrected dataset")
print("  3. model2_test_results.csv - Test set performance")
print("  4. model2_cv_results.csv - Cross-validation results")
print("  5. model2_spatial_cv_results.csv - Spatial CV results")
print("  6. viz1_model2_performance_comparison.png - Model comparison")
print("  7. viz2_model2_predictions.png - Predicted vs actual")
print("  8. viz3_model2_feature_importance.png - Feature importance chart")
print("  9. viz4_model2_spatial_cv.png - Spatial generalization")
print(" 10. viz5_model2_correction_impact.png - Before/after correction")
print(" 11. viz6_model2_residuals.png - Residuals analysis")

print("\n✓ Model 2 analysis complete!")
print("\n" + "="*80)


--------------------------------------------------------------------------------
STEP 8: APPLYING ML CORRECTIONS
--------------------------------------------------------------------------------

Using Random Forest for corrections

Original ERA5 RMSE: 2.048°C
ML-Corrected RMSE: 1.459°C
Improvement: +28.7%

--------------------------------------------------------------------------------
STEP 9: CREATING VISUALIZATIONS
--------------------------------------------------------------------------------

Creating Viz 1: Model Performance Comparison...
  ✓ Saved: viz1_model2_performance_comparison.png

Creating Viz 2: Predicted vs Actual Errors...
  ✓ Saved: viz2_model2_predictions.png

Creating Viz 3: Feature Importance...
  ✓ Saved: viz3_model2_feature_importance.png

Creating Viz 4: Spatial Cross-Validation...
  ✓ Saved: viz4_model2_spatial_cv.png

Creating Viz 5: Before/After ML Correction...
  ✓ Saved: viz5_model2_correction_impact.png

Creating Viz 6: Residuals Analysis...
  ✓ Saved: vi

## Model 3: Rigorous statistical approach with interpretable coefficients:
  - Linear Mixed Effects Model (city as random effect)
  - Multiple Linear Regression (baseline)
  - Quantile Regression (robust to outliers)
  - Statistical inference (p-values, confidence intervals)
  - Model diagnostics (assumptions testing)
  
Goal: Scientifically rigorous, interpretable, publishable model


In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Statistical modeling libraries
try:
    import statsmodels.api as sm
    import statsmodels.formula.api as smf
    from statsmodels.regression.mixed_linear_model import MixedLM
    statsmodels_available = True
except ImportError:
    print("⚠️  Warning: statsmodels not available. Installing...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'statsmodels', '-q'])
    import statsmodels.api as sm
    import statsmodels.formula.api as smf
    from statsmodels.regression.mixed_linear_model import MixedLM
    statsmodels_available = True

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 11

# ============================================================================
# LOAD DATA
# ============================================================================

WEEK3_DIR ="/content/drive/MyDrive/GenHack2025/week3_quantitative_metrics"
WEEK4_DIR = "week4_explanatory_model3"
os.makedirs(WEEK4_DIR, exist_ok=True)

print("="*80)
print("WEEK 4 - MODEL 3: STATISTICAL REGRESSION MODEL")
print("="*80)

# Load master dataset
df = pd.read_csv(f"{WEEK3_DIR}/master_dataset.csv")
print(f"\nLoaded {len(df)} stations from Week 3")

# ============================================================================
# STEP 1: DATA PREPARATION
# ============================================================================

print("\n" + "-"*80)
print("STEP 1: DATA PREPARATION FOR REGRESSION")
print("-"*80)

# Get dominant season
def get_dominant_season(row):
    seasons = []
    for season in ['Winter', 'Spring', 'Summer', 'Fall']:
        if f'n_days_{season}' in row.index and row[f'n_days_{season}'] > 0:
            seasons.append((season, row[f'n_days_{season}']))
    if seasons:
        return max(seasons, key=lambda x: x[1])[0]
    return 'Summer'

df['season'] = df.apply(get_dominant_season, axis=1)

# Prepare regression dataset
regression_vars = [
    'bias',  # Target
    'ndvi_mean',
    'elevation',
    'lat',
    'lon',
    'distance_to_city_km',
    'distance_to_coast_km',
    'mean_station_temp',
    'mean_era5_temp', # ADDED THIS LINE
    'season',
    'city',
    'category'
]

df_reg = df[regression_vars].dropna()
print(f"\n✓ Clean regression dataset: {len(df_reg)} observations")

# Standardize continuous predictors for interpretation
continuous_vars = ['elevation', 'lat', 'lon', 'distance_to_city_km',
                   'distance_to_coast_km', 'mean_station_temp', 'ndvi_mean']

print("\nStandardizing continuous variables for interpretation...")
for var in continuous_vars:
    mean_val = df_reg[var].mean()
    std_val = df_reg[var].std()
    df_reg[f'{var}_std'] = (df_reg[var] - mean_val) / std_val
    print(f"  {var:<25}: mean={mean_val:.2f}, std={std_val:.2f}")

# ============================================================================
# STEP 2: MODEL 1 - SIMPLE LINEAR REGRESSION
# ============================================================================

print("\n" + "-"*80)
print("STEP 2: SIMPLE LINEAR REGRESSION (BASELINE)")
print("-"*80)

# Prepare design matrix
X_simple = df_reg[['elevation_std', 'mean_station_temp_std', 'lat_std',
                    'lon_std', 'distance_to_city_km_std', 'distance_to_coast_km_std',
                    'ndvi_mean_std']].copy()

# Add season dummies
season_dummies = pd.get_dummies(df_reg['season'], prefix='season', drop_first=True)
# Convert boolean dummies to integer type (0 or 1) to avoid ValueError in statsmodels
season_dummies = season_dummies.astype(int)
X_simple = pd.concat([X_simple, season_dummies], axis=1)

# Add constant
X_simple = sm.add_constant(X_simple)

y = df_reg['bias']

# Fit OLS model
print("\nFitting Ordinary Least Squares model...")
ols_model = sm.OLS(y, X_simple).fit()

print("\n" + "="*60)
print("MODEL 1: SIMPLE LINEAR REGRESSION RESULTS")
print("="*60)
print(ols_model.summary())

# Save summary
with open(f"{WEEK4_DIR}/model3_ols_summary.txt", 'w') as f:
    f.write(str(ols_model.summary()))
print(f"\n✓ Saved OLS summary: model3_ols_summary.txt")

# Extract key statistics
ols_results = {
    'Model': 'OLS',
    'R2': ols_model.rsquared,
    'Adj_R2': ols_model.rsquared_adj,
    'AIC': ols_model.aic,
    'BIC': ols_model.bic,
    'RMSE': np.sqrt(ols_model.mse_resid),
    'N_obs': int(ols_model.nobs)
}

print(f"\n📊 Model Statistics:")
print(f"  R²: {ols_results['R2']:.4f}")
print(f"  Adjusted R²: {ols_results['Adj_R2']:.4f}")
print(f"  RMSE: {ols_results['RMSE']:.3f}°C")
print(f"  AIC: {ols_results['AIC']:.2f}")
print(f"  BIC: {ols_results['BIC']:.2f}")

# ============================================================================
# STEP 3: MODEL 2 - LINEAR MIXED EFFECTS MODEL
# ============================================================================

print("\n" + "-"*80)
print("STEP 3: LINEAR MIXED EFFECTS MODEL (HIERARCHICAL)")
print("-"*80)

print("\nFitting Mixed Effects Model with city as random effect...")
print("  Fixed effects: elevation, temperature, lat, lon, distances, NDVI, season")
print("  Random effect: city (accounts for city-to-city variation)")

# Prepare formula
formula = """bias ~ elevation_std + mean_station_temp_std + lat_std + lon_std +
             distance_to_city_km_std + distance_to_coast_km_std + ndvi_mean_std +
             C(season)"""

# Fit mixed model
try:
    mixed_model = smf.mixedlm(formula, df_reg, groups=df_reg["city"]).fit()

    print("\n" + "="*60)
    print("MODEL 2: LINEAR MIXED EFFECTS MODEL RESULTS")
    print("="*60)
    print(mixed_model.summary())

    # Save summary
    with open(f"{WEEK4_DIR}/model3_mixed_summary.txt", 'w') as f:
        f.write(str(mixed_model.summary()))
    print(f"\n✓ Saved Mixed Model summary: model3_mixed_summary.txt")

    mixed_results = {
        'Model': 'Mixed Effects',
        'AIC': mixed_model.aic,
        'BIC': mixed_model.bic,
        'Log_Likelihood': mixed_model.llf,
        'N_obs': int(mixed_model.nobs)
    }

    print(f"\n📊 Model Statistics:")
    print(f"  AIC: {mixed_results['AIC']:.2f}")
    print(f"  BIC: {mixed_results['BIC']:.2f}")
    print(f"  Log-Likelihood: {mixed_results['Log_Likelihood']:.2f}")

    # Random effects variance
    print(f"\n🎲 Random Effects:")
    print(f"  City variance: {mixed_model.cov_re.iloc[0,0]:.4f}")
    print(f"  Residual variance: {mixed_model.scale:.4f}")

    mixed_available = True

except Exception as e:
    print(f"\n⚠️  Mixed model fitting failed: {e}")
    print("   Continuing with OLS results only...")
    mixed_available = False
    mixed_model = None

# ============================================================================
# STEP 4: MODEL 3 - QUANTILE REGRESSION (ROBUST)
# ============================================================================

print("\n" + "-"*80)
print("STEP 4: QUANTILE REGRESSION (ROBUST TO OUTLIERS)")
print("-"*80)

print("\nFitting Quantile Regression at median (50th percentile)...")
print("  (Robust alternative to OLS, less sensitive to outliers)")

try:
    # Prepare data
    X_quant = X_simple.copy()

    # Fit quantile regression at median
    qr_model = sm.QuantReg(y, X_quant).fit(q=0.5)

    print("\n" + "="*60)
    print("MODEL 3: QUANTILE REGRESSION RESULTS (Median)")
    print("="*60)
    print(qr_model.summary())

    # Save summary
    with open(f"{WEEK4_DIR}/model3_quantreg_summary.txt", 'w') as f:
        f.write(str(qr_model.summary()))
    print(f"\n✓ Saved Quantile Regression summary: model3_quantreg_summary.txt")

    quantreg_available = True

except Exception as e:
    print(f"\n⚠️  Quantile regression failed: {e}")
    quantreg_available = False
    qr_model = None

# ============================================================================
# STEP 5: COEFFICIENT COMPARISON & INTERPRETATION
# ============================================================================

print("\n" + "-"*80)
print("STEP 5: COEFFICIENT COMPARISON & INTERPRETATION")
print("-"*80)

# Extract coefficients
def extract_coefficients(model, model_name):
    coefs = []
    for param in model.params.index:
        if param != 'const' and not param.startswith('Group'):
            coef_val = model.params[param]
            std_err = model.bse[param]
            p_val = model.pvalues[param]
            ci_lower = model.conf_int().loc[param, 0]
            ci_upper = model.conf_int().loc[param, 1]

            # Significance stars
            if p_val < 0.001:
                sig = '***'
            elif p_val < 0.01:
                sig = '**'
            elif p_val < 0.05:
                sig = '*'
            else:
                sig = ''

            coefs.append({
                'Model': model_name,
                'Variable': param,
                'Coefficient': coef_val,
                'Std_Error': std_err,
                'P_Value': p_val,
                'CI_Lower': ci_lower,
                'CI_Upper': ci_upper,
                'Significance': sig
            })
    return coefs

# Collect coefficients from all models
all_coefs = []
all_coefs.extend(extract_coefficients(ols_model, 'OLS'))

if mixed_available:
    all_coefs.extend(extract_coefficients(mixed_model, 'Mixed Effects'))

if quantreg_available:
    all_coefs.extend(extract_coefficients(qr_model, 'Quantile Reg'))

df_coefs = pd.DataFrame(all_coefs)

# Save coefficients
df_coefs.to_csv(f"{WEEK4_DIR}/model3_coefficients_comparison.csv", index=False)
print(f"\n✓ Saved coefficient comparison: model3_coefficients_comparison.csv")

# Print key coefficients
print("\n📊 KEY COEFFICIENTS (OLS Model):")
print(f"{'Variable':<30} {'Coef':<10} {'Std Err':<10} {'P-value':<10} {'Sig':<5}")
print("-" * 70)

key_vars = ['elevation_std', 'mean_station_temp_std', 'ndvi_mean_std',
            'distance_to_city_km_std', 'lat_std']

for var in key_vars:
    if var in ols_model.params.index:
        coef = ols_model.params[var]
        se = ols_model.bse[var]
        pval = ols_model.pvalues[var]

        if pval < 0.001:
            sig = '***'
        elif pval < 0.01:
            sig = '**'
        elif pval < 0.05:
            sig = '*'
        else:
            sig = ''

        print(f"   {var:<30} {coef:>+8.4f}  {se:>8.4f}  {pval:>8.6f}  {sig:<5}")

print("\nSignificance codes: *** p<0.001, ** p<0.01, * p<0.05")

# ============================================================================
# STEP 6: MODEL DIAGNOSTICS
# ============================================================================

print("\n" + "-"*80)
print("STEP 6: MODEL DIAGNOSTICS")
print("-"*80)

# Residuals
residuals = ols_model.resid
fitted = ols_model.fittedvalues

# Normality test
print("\n1. Normality Test (Jarque-Bera):")
jb_stat, jb_pval = stats.jarque_bera(residuals)
print(f"   Test statistic: {jb_stat:.4f}")
print(f"   P-value: {jb_pval:.6f}")
if jb_pval > 0.05:
    print("   ✓ Residuals appear normally distributed")
else:
    print("   ⚠️  Residuals may not be perfectly normal (but OLS is robust)")

# Homoscedasticity test (Breusch-Pagan)
print("\n2. Homoscedasticity Test (Breusch-Pagan):")
from statsmodels.stats.diagnostic import het_breuschpagan
bp_stat, bp_pval, _, _ = het_breuschpagan(residuals, X_simple)
print(f"   Test statistic: {bp_stat:.4f}")
print(f"   P-value: {bp_pval:.6f}")
if bp_pval > 0.05:
    print("   ✓ Homoscedasticity assumption satisfied")
else:
    print("   ⚠️  Some heteroscedasticity detected (common in real data)")

# Multicollinearity (VIF)
print("\n3. Multicollinearity Check (VIF):")
from statsmodels.stats.outliers_influence import variance_inflation_factor

print(f"   {'Variable':<30} {'VIF':<10} {'Status':<20}")
print("   " + "-" * 60)

vif_data = []
for i, col in enumerate(X_simple.columns):
    if col != 'const':
        vif = variance_inflation_factor(X_simple.values, i)
        vif_data.append({'Variable': col, 'VIF': vif})

        if vif < 5:
            status = '✓ Low'
        elif vif < 10:
            status = '⚠️  Moderate'
        else:
            status = '❌ High'

        print(f"   {col:<30} {vif:>8.2f}  {status:<20}")

df_vif = pd.DataFrame(vif_data)
df_vif.to_csv(f"{WEEK4_DIR}/model3_vif.csv", index=False)

# ============================================================================
# STEP 7: PREDICTIONS & CORRECTIONS
# ============================================================================

print("\n" + "-"*80)
print("STEP 7: APPLYING STATISTICAL CORRECTIONS")
print("-"*80)

# Predictions from OLS
df_reg['predicted_bias_ols'] = ols_model.predict(X_simple)
df_reg['era5_corrected_ols'] = df_reg['mean_era5_temp'] - df_reg['predicted_bias_ols']
df_reg['error_corrected_ols'] = df_reg['era5_corrected_ols'] - df_reg['mean_station_temp']

# Original performance
original_rmse = df['rmse'].mean()
ols_rmse = np.sqrt((df_reg['error_corrected_ols']**2).mean())
ols_improvement = ((original_rmse - ols_rmse) / original_rmse) * 100

print(f"\n📊 OLS Correction Performance:")
print(f"  Original ERA5 RMSE: {original_rmse:.3f}°C")
print(f"  OLS-Corrected RMSE: {ols_rmse:.3f}°C")
print(f"  Improvement: {ols_improvement:+.1f}%")

# Mixed model predictions (if available)
if mixed_available:
    df_reg['predicted_bias_mixed'] = mixed_model.fittedvalues
    df_reg['era5_corrected_mixed'] = df_reg['mean_era5_temp'] - df_reg['predicted_bias_mixed']
    df_reg['error_corrected_mixed'] = df_reg['era5_corrected_mixed'] - df_reg['mean_station_temp']

    mixed_rmse = np.sqrt((df_reg['error_corrected_mixed']**2).mean())
    mixed_improvement = ((original_rmse - mixed_rmse) / original_rmse) * 100

    print(f"\n📊 Mixed Model Correction Performance:")
    print(f"  Mixed-Corrected RMSE: {mixed_rmse:.3f}°C")
    print(f"  Improvement: {mixed_improvement:+.1f}%")

# Save corrected dataset
df_reg.to_csv(f"{WEEK4_DIR}/model3_corrected_dataset.csv", index=False)
print(f"\n✓ Saved corrected dataset: model3_corrected_dataset.csv")

# ============================================================================
# VISUALIZATIONS
# ============================================================================

print("\n" + "-"*80)
print("STEP 8: CREATING VISUALIZATIONS")
print("-"*80)

# Viz 1: Coefficient Plot with Confidence Intervals
print("\nCreating Viz 1: Coefficient Plot...")

fig, ax = plt.subplots(figsize=(12, 10))

# Get OLS coefficients (exclude intercept and season dummies for clarity)
coef_plot_vars = ['elevation_std', 'mean_station_temp_std', 'lat_std', 'lon_std',
                  'distance_to_city_km_std', 'distance_to_coast_km_std', 'ndvi_mean_std']

plot_data = []
for var in coef_plot_vars:
    if var in ols_model.params.index:
        plot_data.append({
            'Variable': var.replace('_std', ''),
            'Coefficient': ols_model.params[var],
            'CI_Lower': ols_model.conf_int().loc[var, 0],
            'CI_Upper': ols_model.conf_int().loc[var, 1],
            'P_Value': ols_model.pvalues[var]
        })

df_plot = pd.DataFrame(plot_data).sort_values('Coefficient')

y_pos = np.arange(len(df_plot))
colors = ['#e74c3c' if p < 0.05 else '#95a5a6' for p in df_plot['P_Value']]

ax.barh(y_pos, df_plot['Coefficient'], xerr=[
    df_plot['Coefficient'] - df_plot['CI_Lower'],
    df_plot['CI_Upper'] - df_plot['Coefficient']
], color=colors, alpha=0.7, edgecolor='black', linewidth=1.5,
capsize=5, error_kw={'linewidth': 2})

ax.axvline(0, color='black', linestyle='--', linewidth=2, alpha=0.7)
ax.set_yticks(y_pos)
ax.set_yticklabels(df_plot['Variable'])
ax.set_xlabel('Standardized Coefficient (°C per SD)', fontsize=13)
ax.set_title('Model 3: OLS Regression Coefficients with 95% CI\n(Red = Significant at p<0.05)',
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# Add significance markers
for i, (coef, pval) in enumerate(zip(df_plot['Coefficient'], df_plot['P_Value'])):
    if pval < 0.001:
        sig = '***'
    elif pval < 0.01:
        sig = '**'
    elif pval < 0.05:
        sig = '*'
    else:
        sig = ''

    if sig:
        ax.text(coef + 0.02, i, sig, va='center', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(f"{WEEK4_DIR}/viz1_model3_coefficients.png", dpi=300, bbox_inches='tight')
plt.close()
print("  ✓ Saved: viz1_model3_coefficients.png")

# Viz 2: Diagnostic Plots (4-panel)
print("\nCreating Viz 2: Diagnostic Plots...")

fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# Panel A: Residuals vs Fitted
ax = axes[0, 0]
ax.scatter(fitted, residuals, alpha=0.5, s=30, c='steelblue', edgecolors='black', linewidth=0.5)
ax.axhline(0, color='red', linestyle='--', linewidth=2)
ax.set_xlabel('Fitted Values (°C)', fontsize=11)
ax.set_ylabel('Residuals (°C)', fontsize=11)
ax.set_title('A) Residuals vs Fitted\n(Check for heteroscedasticity)', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Panel B: Q-Q Plot
ax = axes[0, 1]
stats.probplot(residuals, dist="norm", plot=ax)
ax.set_title('B) Normal Q-Q Plot\n(Check normality assumption)', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Panel C: Scale-Location
ax = axes[1, 0]
standardized_resid = residuals / np.std(residuals)
ax.scatter(fitted, np.sqrt(np.abs(standardized_resid)), alpha=0.5, s=30,
          c='coral', edgecolors='black', linewidth=0.5)
ax.set_xlabel('Fitted Values (°C)', fontsize=11)
ax.set_ylabel('√|Standardized Residuals|', fontsize=11)
ax.set_title('C) Scale-Location Plot\n(Check homoscedasticity)', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Panel D: Residuals Histogram
ax = axes[1, 1]
ax.hist(residuals, bins=50, alpha=0.7, color='steelblue', edgecolor='black', density=True)

# Overlay normal distribution
mu, sigma = residuals.mean(), residuals.std()
x = np.linspace(residuals.min(), residuals.max(), 100)
ax.plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal Distribution')

ax.axvline(0, color='black', linestyle='--', linewidth=2)
ax.set_xlabel('Residuals (°C)', fontsize=11)
ax.set_ylabel('Density', fontsize=11)
ax.set_title(f'D) Residual Distribution\nMean: {mu:+.4f}°C, Std: {sigma:.4f}°C',
             fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

fig.suptitle('Model 3: OLS Regression Diagnostics', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.savefig(f"{WEEK4_DIR}/viz2_model3_diagnostics.png", dpi=300, bbox_inches='tight')
plt.close()
print("  ✓ Saved: viz2_model3_diagnostics.png")

# Viz 3: Before vs After Correction
print("\nCreating Viz 3: Before/After Statistical Correction...")

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Original
ax = axes[0]
scatter = ax.scatter(df_reg['mean_station_temp'], df_reg['mean_era5_temp'],
                    c=df_reg['elevation'], cmap='terrain', s=50, alpha=0.6,
                    edgecolors='black', linewidth=0.5)

min_val = min(df_reg['mean_station_temp'].min(), df_reg['mean_era5_temp'].min())
max_val = max(df_reg['mean_station_temp'].max(), df_reg['mean_era5_temp'].max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2,
        label='Perfect Agreement', alpha=0.7)

ax.set_xlabel('Observed Temperature (°C)', fontsize=12)
ax.set_ylabel('ERA5 Temperature (°C)', fontsize=12)
ax.set_title(f'Original ERA5\nRMSE: {original_rmse:.3f}°C',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# OLS Corrected
ax = axes[1]
scatter = ax.scatter(df_reg['mean_station_temp'], df_reg['era5_corrected_ols'],
                    c=df_reg['elevation'], cmap='terrain', s=50, alpha=0.6,
                    edgecolors='black', linewidth=0.5)

min_val = min(df_reg['mean_station_temp'].min(), df_reg['era5_corrected_ols'].min())
max_val = max(df_reg['mean_station_temp'].max(), df_reg['era5_corrected_ols'].max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2,
        label='Perfect Agreement', alpha=0.7)

ax.set_xlabel('Observed Temperature (°C)', fontsize=12)
ax.set_ylabel('Corrected ERA5 Temperature (°C)', fontsize=12)
ax.set_title(f'OLS-Corrected ERA5\nRMSE: {ols_rmse:.3f}°C ({ols_improvement:+.1f}%)',
             fontsize=13, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

fig.colorbar(scatter, ax=axes, label='Elevation (m)', orientation='horizontal', pad=0.1)

fig.suptitle('Model 3: Impact of Statistical Regression Correction',
             fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(f"{WEEK4_DIR}/viz3_model3_correction_impact.png", dpi=300, bbox_inches='tight')
plt.close()
print("  ✓ Saved: viz3_model3_correction_impact.png")

# Viz 4: Model Comparison (if multiple models available)
if mixed_available:
    print("\nCreating Viz 4: Model Comparison...")

    fig, ax = plt.subplots(figsize=(10, 7))

    models_comparison = [
        {'Model': 'OLS', 'RMSE': ols_rmse, 'R2': ols_results['R2']},
    ]

    if mixed_available:
        models_comparison.append({
            'Model': 'Mixed Effects',
            'RMSE': mixed_rmse,
            'R2': np.nan  # Mixed models don't have standard R²
        })

    df_comparison = pd.DataFrame(models_comparison)

    x_pos = np.arange(len(df_comparison))
    colors = ['#3498db', '#e74c3c'][:len(df_comparison)]

    bars = ax.bar(x_pos, df_comparison['RMSE'], color=colors, alpha=0.7,
                  edgecolor='black', linewidth=1.5)

    # Add original RMSE line
    ax.axhline(original_rmse, color='red', linestyle='--', linewidth=2,
              label=f'Original: {original_rmse:.3f}°C', alpha=0.7)

    # Add values
    for bar, rmse in zip(bars, df_comparison['RMSE']):
        height = bar.get_height()
        improvement = ((original_rmse - rmse) / original_rmse) * 100
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.05,
                f'{rmse:.3f}°C\n({improvement:+.1f}%)',
                ha='center', va='bottom', fontsize=11, fontweight='bold')

    ax.set_xticks(x_pos)
    ax.set_xticklabels(df_comparison['Model'])
    ax.set_ylabel('RMSE (°C)', fontsize=13)
    ax.set_title('Model 3: Statistical Model Comparison', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.savefig(f"{WEEK4_DIR}/viz4_model3_comparison.png", dpi=300, bbox_inches='tight')
    plt.close()
    print("  ✓ Saved: viz4_model3_comparison.png")

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*80)
print("MODEL 3: STATISTICAL REGRESSION MODEL - COMPLETE")
print("="*80)

print("\n📊 MODEL 1: ORDINARY LEAST SQUARES")
print(f"  R²: {ols_results['R2']:.4f}")
print(f"  Adjusted R²: {ols_results['Adj_R2']:.4f}")
print(f"  RMSE: {ols_results['RMSE']:.3f}°C")
print(f"  AIC: {ols_results['AIC']:.1f}")
print(f"  Observations: {ols_results['N_obs']}")

if mixed_available:
    print("\n📊 MODEL 2: LINEAR MIXED EFFECTS")
    print(f"  AIC: {mixed_results['AIC']:.1f}")
    print(f"  BIC: {mixed_results['BIC']:.1f}")
    print(f"  Log-Likelihood: {mixed_results['Log_Likelihood']:.2f}")

print("\n🎯 CORRECTION PERFORMANCE:")
print(f"  Original ERA5 RMSE: {original_rmse:.3f}°C")
print(f"  OLS-Corrected RMSE: {ols_rmse:.3f}°C")
print(f"  Improvement: {ols_improvement:+.1f}%")

if mixed_available:
    print(f"  Mixed-Corrected RMSE: {mixed_rmse:.3f}°C")
    print(f"  Improvement: {mixed_improvement:+.1f}%")

print("\n🔬 MODEL DIAGNOSTICS:")
print(f"  Normality (Jarque-Bera): {'✓ Pass' if jb_pval > 0.05 else '⚠️  Borderline'}")
print(f"  Homoscedasticity (BP): {'✓ Pass' if bp_pval > 0.05 else '⚠️  Some heteroscedasticity'}")
print(f"  Multicollinearity: {'✓ Low VIF' if all(df_vif['VIF'] < 10) else '⚠️  Some collinearity'}")

print("\n🏆 MOST SIGNIFICANT PREDICTORS:")
significant_vars = df_coefs[(df_coefs['Model'] == 'OLS') & (df_coefs['P_Value'] < 0.05)].sort_values('P_Value')
for i, row in significant_vars.head(5).iterrows():
    print(f"  {i+1}. {row['Variable']:<30} (coef={row['Coefficient']:+.4f}, p={row['P_Value']:.6f})")

print("\n📁 OUTPUTS SAVED:")
print("  1. model3_ols_summary.txt - Full OLS regression output")
if mixed_available:
    print("  2. model3_mixed_summary.txt - Mixed effects model output")
if quantreg_available:
    print("  3. model3_quantreg_summary.txt - Quantile regression output")
print("  4. model3_coefficients_comparison.csv - All model coefficients")
print("  5. model3_vif.csv - Multicollinearity diagnostics")
print("  6. model3_corrected_dataset.csv - Corrected temperatures")
print("  7. viz1_model3_coefficients.png - Coefficient plot with CI")
print("  8. viz2_model3_diagnostics.png - 4-panel diagnostics")
print("  9. viz3_model3_correction_impact.png - Before/after correction")
if mixed_available:
    print(" 10. viz4_model3_comparison.png - Model comparison")

print("\n💡 INTERPRETATION GUIDE:")
print("  • Standardized coefficients show effect per 1 SD change in predictor")
print("  • Positive coefficient = increases bias (ERA5 becomes warmer)")
print("  • Negative coefficient = decreases bias (ERA5 becomes colder)")
print("  • P-values test if effect is statistically different from zero")
print("  • R² shows proportion of variance explained by the model")
print("  • Mixed effects account for city-level clustering")

print("\n✓ Model 3 analysis complete!")
print("\n" + "="*80)


WEEK 4 - MODEL 3: STATISTICAL REGRESSION MODEL

Loaded 286 stations from Week 3

--------------------------------------------------------------------------------
STEP 1: DATA PREPARATION FOR REGRESSION
--------------------------------------------------------------------------------

✓ Clean regression dataset: 286 observations

Standardizing continuous variables for interpretation...
  elevation                : mean=457.71, std=450.81
  lat                      : mean=46.52, std=4.87
  lon                      : mean=7.37, std=8.09
  distance_to_city_km      : mean=92.01, std=42.72
  distance_to_coast_km     : mean=556.93, std=193.03
  mean_station_temp        : mean=17.67, std=2.99
  ndvi_mean                : mean=0.44, std=0.15

--------------------------------------------------------------------------------
STEP 2: SIMPLE LINEAR REGRESSION (BASELINE)
--------------------------------------------------------------------------------

Fitting Ordinary Least Squares model...

MODEL 1: 